In [27]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

train_dir = os.path.join('chest_xray/train/')
eval_dir = os.path.join('chest_xray/test/')

In [35]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255,zoom_range=0.2,shear_range=0.2,rotation_range = 20,fill_mode = 'constant')

train_generator = train_datagen.flow_from_directory(
  train_dir,
  batch_size = 30,
  target_size=(256, 256),
  class_mode='binary'
)

Found 5216 images belonging to 2 classes.


In [36]:
# All images will be rescaled by 1./255
eval_datagen = ImageDataGenerator(rescale=1/255)

eval_generator = eval_datagen.flow_from_directory(
  eval_dir,
  target_size=(256, 256),
  class_mode='binary'
)

Found 624 images belonging to 2 classes.


In [47]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(3, 3),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(64, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [48]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_58 (Conv2D)          (None, 254, 254, 64)      1792      
                                                                 
 max_pooling2d_57 (MaxPoolin  (None, 84, 84, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_59 (Conv2D)          (None, 82, 82, 64)        36928     
                                                                 
 max_pooling2d_58 (MaxPoolin  (None, 41, 41, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_60 (Conv2D)          (None, 39, 39, 64)        36928     
                                                                 
 max_pooling2d_59 (MaxPoolin  (None, 19, 19, 64)     

In [49]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [50]:
batch_size=100
history = model.fit_generator(
      train_generator,
      steps_per_epoch=170,
        
      #batch_size=100,
     
      epochs=10,
      validation_data=eval_generator,
      verbose=1)

/var/folders/k3/gcflkw4j1lx_kn_v5p132mym0000gn/T/ipykernel_6146/995759766.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10


2022-05-03 14:18:52.040649: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


170/170 [==============================] - ETA: 0s - loss: 0.5718 - accuracy: 0.7522

2022-05-03 14:19:49.621922: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


170/170 [==============================] - 62s 352ms/step - loss: 0.5718 - accuracy: 0.7522 - val_loss: 0.5143 - val_accuracy: 0.6731
Epoch 2/10
170/170 [==============================] - 60s 352ms/step - loss: 0.3329 - accuracy: 0.8631 - val_loss: 0.3289 - val_accuracy: 0.8638
Epoch 3/10
170/170 [==============================] - 60s 352ms/step - loss: 0.2637 - accuracy: 0.8972 - val_loss: 0.2720 - val_accuracy: 0.8894
Epoch 4/10
170/170 [==============================] - 60s 352ms/step - loss: 0.2133 - accuracy: 0.9152 - val_loss: 0.3813 - val_accuracy: 0.8510
Epoch 5/10
170/170 [==============================] - 60s 352ms/step - loss: 0.1889 - accuracy: 0.9296 - val_loss: 0.7301 - val_accuracy: 0.6554
Epoch 6/10
170/170 [==============================] - 60s 351ms/step - loss: 0.1705 - accuracy: 0.9368 - val_loss: 0.2335 - val_accuracy: 0.9103
Epoch 7/10
170/170 [==============================] - 60s 353ms/step - loss: 0.1574 - accuracy: 0.9460 - val_loss: 0.2781 - val_accuracy: 0.8

In [52]:
test_acc = model.evaluate(eval_generator,batch_size=100, verbose=1) 

print('Test accuracy:', test_acc)

20/20 [==============================] - 3s 156ms/step - loss: 0.2629 - accuracy: 0.8990
Test accuracy: [0.2628757655620575, 0.8990384936332703]


In [63]:
from keras.preprocessing import image



img =  image.load_img('norm.jpeg',target_size=(256,256))

x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)


images = np.vstack([x])
classes = model.predict(images,batch_size=10)

print (classes)
print (classes[0])


[[0.]]
[0.]
